In [173]:
baseurl = "https://www.valueresearchonline.com/api/stocks/key-ratio-chart/2933"
headers = {}
headers= "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"

import sys
import json
import datetime
import urllib
import numpy as np
import datetime
import pandas as pd
from datetime import timedelta
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int(round((dt - epoch).total_seconds() * 1000.0, 0))


In [166]:
prices = pd.read_csv ('prices_small.csv')
max_price_date = prices.Date.max()

In [167]:

# prices_small = prices[prices.Date >= max_price_date]
# prices_small.shape
# prices_small.to_csv('prices_small.csv', index=False)

In [169]:
prices[(prices['NAME OF COMPANY'].str.startswith('Pidi')) & (prices.Date >=max_price_date)]

,100D Change,10D Change,150D Change,1D Change,200D Change,20D Change,30D Change,50D Change,5D Change,Close,Date,NAME OF COMPANY,SYMBOL,Volume
1056,0.453732,5.913264,-2.02053,3.617603,8.731528,-17.624811,-13.179042,-7.347693,-0.434977,1350.5,2020-04-09,Pidilite Industries Limited,PIDILITIND,293300.0


In [170]:
file_name = 'vro_revenues.csv'
all_data = pd.read_csv(file_name)
all_data = all_data[~all_data['Stock Name'].str.startswith('S&P')]

In [171]:
all_data['1y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(1)
all_data['2y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(2)
all_data['3y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(3)
all_data = all_data[all_data['1y_change']>0]
all_data = all_data[all_data['2y_change']>0]
all_data = all_data[all_data.name != 'RoE']
all_data.dropna(inplace=True)
all_data.shape

(5713, 9)

In [172]:
all_data[all_data['Stock Name'].str.startswith('Pidil') ]

,Date,Stock Name,name,ticker_id,value,yearly_change,1y_change,2y_change,3y_change
56619,2019-03-01,Pidilite Industries,Revenue,2933.0,7078.74,0.164571,0.164571,0.260282,0.320372
56620,2019-12-01,Pidilite Industries,Revenue,2933.0,7388.91,0.043817,0.043817,0.215599,0.315504
56625,2019-12-01,Pidilite Industries,Earnings,2933.0,23.50,0.290500,0.290500,0.240106,0.401312
75608,2019-03-01,Pidilite Industries,Cash From Operations,2933.0,844.78,0.060708,0.060708,0.064290,-0.069082


In [174]:
max_revenue = all_data[all_data.name=='Revenue'].value.max()
max_cash = all_data[all_data.name=='Cash From Operations'].value.max()
max_vro_cash_flow_date = all_data[all_data.name == 'Cash From Operations']['Date'].max()
max_vro_revenue_date = all_data[all_data.name == 'Revenue']['Date'].max()
max_cash, max_revenue, max_vro_cash_flow_date

(71459.0, 611934.0, '2019-12-01')

In [181]:
@interact
def filter_cash_flow(cash_flow_change = (2,10), revenue_1y_change = (2,10), min_revenue = (4000, 15000, 500)):
    stocks_by_cash_change = all_data.loc[(all_data.name == 'Cash From Operations') & (all_data['1y_change'] > cash_flow_change/100) & (all_data.Date >= '2019-03-01') ]['Stock Name'].unique().tolist()
    stocks_by_rev_change = all_data.loc[(all_data.name == 'Revenue') & (all_data['1y_change'] > revenue_1y_change/100) & (all_data.Date >= '2019-12-01')]['Stock Name'].unique().tolist()
    stock_with_min_revenue = all_data.loc[(all_data.name == 'Revenue') & (all_data.value >= min_revenue)]['Stock Name'].unique().tolist()
    stocks = set(stocks_by_cash_change).intersection(set(stocks_by_rev_change)).intersection(set(stock_with_min_revenue))
    df= all_data.loc[(all_data['Stock Name'].isin(stocks)) & (all_data.Date == max_vro_date) ]
    df = df.loc[(df.name == 'Revenue')]
    prices_df = pd.DataFrame()
    for stock in stocks:
        p = prices.loc[(prices['NAME OF COMPANY'].str.startswith(stock)) & (prices.Date >=max_price_date)]
        if len(p)>0:
            p['Stock Name'] = stock
            prices_df = pd.concat([prices_df, p])

    df = df.merge(prices_df, on='Stock Name', how = 'left')
    df = df.drop(['100D Change', '200D Change', '150D Change', 'yearly_change', 'value', 'NAME OF COMPANY', '3y_change', 'ticker_id', 'SYMBOL', 'Date_x'], axis=1)
    return df.sort_values(['5D Change'])


interactive(children=(IntSlider(value=6, description='cash_flow_change', max=10, min=2), IntSlider(value=6, de…

In [100]:
@interact
def filter_cash_flow(revenue = (0, 30000, 100), cash = (0, 10000, 100), revenue_1y_change = (2,20) ):
    stocks_by_revenue = all_data.loc[(all_data.name == 'Revenue') & (all_data.value > revenue) & (all_data.Date == '2019-12-01')]['Stock Name'].unique().tolist()
    print(len(stocks_by_revenue))
    stocks_by_cash = all_data.loc[(all_data.name == 'Cash From Operations') & (all_data.value > cash) ]['Stock Name'].unique().tolist()
    stocks_by_rev_change = all_data.loc[(all_data.name == 'Revenue') & (all_data['1y_change'] > revenue_1y_change/100) & (all_data.Date == '2019-12-01')]['Stock Name'].unique().tolist()
    print(len(stocks_by_rev_change))
    stocks = set(stocks_by_revenue).intersection(set(stocks_by_cash)).intersection(set(stocks_by_rev_change))
    print(len(stocks))
    df= all_data[(all_data['Stock Name'].isin(stocks)) & (all_data.Date == max_vro_date) ]
#     df = df[(df.name == 'Cash From Operations') & (df.value > cash)]
#     df = all_data[(all_data['Stock Name'].isin(stocks)) & (all_data.Date == '2019-12-01')]
#     df = df.pivot_table('1y_change', index='Stock Name', columns='name')
    prices_df = pd.DataFrame()
    for stock in stocks:
        p = prices[(prices['NAME OF COMPANY'].str.startswith(stock)) & (prices.Date >=max_price_date)]
        prices_df = pd.concat([prices_df, p])
    
    return prices_df

interactive(children=(IntSlider(value=15000, description='revenue', max=30000, step=100), IntSlider(value=5000…